In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import cv2
import numpy as np
import pickle
from keras.datasets import cifar10

In [2]:
!pip install opencv-contrib-python==4.4.0.44

In [3]:
(train_images,train_labels), (test_images,test_labels) = cifar10.load_data() 

In [4]:
def getDetector(mode):
  detector = 0
  if mode == 'sift': detector = cv2.SIFT_create()
  if mode == 'surf' : detector = cv2.SURF_create()
  if mode =='brisk' : detector = cv2.BRISK_create()
  if mode == 'akaze' :  detector = cv2.AKAZE_create() 
  return detector

In [5]:
def get_features(image, detector):
  keypoints, descriptors = detector.detectAndCompute(image, None)
  return keypoints, descriptors

In [6]:
def gray(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [7]:
detector = getDetector('sift')

In [9]:
train_images = train_images[:10000]

In [10]:
descriptors_lst = []
for img in train_images:
  img = gray(img)
  keypoints, descriptors = get_features(img,detector)
  
  if descriptors is not None: descriptors_lst.extend(descriptors)

In [11]:
def VLAD(X, visualDictionary) : 
    
    predictedLabels = visualDictionary.predict(X)
    centers = visualDictionary.cluster_centers_
    labels = visualDictionary.labels_
    k = visualDictionary.n_clusters

    m,d = X.shape
    V=np.zeros([k,d])
    #computing the differences

    # for all the clusters (visual words)
    for i in range(k):
        # if there is at least one descriptor in that cluster
        if np.sum(predictedLabels==i)>0:
            # add the diferences
            V[i]=np.sum(X[predictedLabels==i,:]-centers[i],axis=0)
    

    V = V.flatten()
    # power normalization, also called square-rooting normalization
    V = np.sign(V)*np.sqrt(np.abs(V))

    # L2 normalization

    V = V/np.sqrt(np.dot(V,V))
    return V

In [12]:
def getVLADDescriptors(images, images_lables, visualDic, detector):
    descriptors = []
    labels = []
    
    count = 0
    for image in images : 
        #     Re-sizing the image
        image = cv2.resize(image, (150, 150), interpolation=cv2.INTER_AREA)
        kp, des = get_features(image,detector)
        if des is not None : 
            v = VLAD(des, visualDic)
            descriptors.append(v)
            labels.append(images_lables[count])
        count += 1
            
            
    descriptors = np.asarray(descriptors)
    return descriptors, labels

In [24]:
from sklearn.cluster import KMeans 

kmeans = KMeans(n_clusters = 64)
kmeans.fit(descriptors_lst)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=64, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [25]:
x_train, y_train = getVLADDescriptors(train_images, train_labels, kmeans, detector)

In [26]:
x_test, y_test = getVLADDescriptors(test_images, test_labels, kmeans, detector)

In [27]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [28]:
y_train = np.ravel(y_train)
y_test = np.ravel(y_test) 

In [29]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [30]:
scaler = StandardScaler()

In [31]:
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [ ]:
clf = SVC(kernel = 'poly', degree = 3, gamma='auto')
clf.fit(x_train, y_train)


print(clf.score(x_test,y_test) * 100)  

In [ ]:
lr = LogisticRegression(penalty = 'l2')
lr.fit(x_train,y_train)
print(lr.score(x_test,y_test) * 100)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(x_train,y_train)
neigh.score(x_test[:2000],y_test[:2000]) * 100